### Eksik Değerlerin Bağımlı Değişken ile Analizi

##### Eksik Değerlerin Bağımlı Değişken ile İlişkisinin İncelenmesi
* AMAÇ: Bir veri setimiz var, bu veri setinde çeşitli değişkenler var, bu değişkenlerde eksiklikler var işte bu eksikliklerin acaba bağımlı değişken tarafında bir karşılığı var mı, acaba bu özel bir yapı mıdır, acaba bağımlı değişkenle ilişkisi olan bazı değişkenleri gözden mi kaçıracağız eğer bunlara özel bir şekilde bakmazsak gibi soruların birlikte yanıtını değerlendiriyor olacağız... 

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import missingno as msno
from matplotlib import pyplot as plt

In [2]:
def load():
    data = pd.read_csv("titanic.csv")
    return data

df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
def missing_values_table(dataframe, na_name=False):
    
    # Sadece eksikliğe sahip kolonların (değişkenlerin) seçilmesi:
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    # Eksik değer sayısı:
    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    
    # Eksik değer oranı:
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    
    # bunları bir dataframe'e çevirme:
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")

    if na_name:
        return na_columns

In [5]:
# Eksik değere sahip olan değişkenleri çekelim....
missing_values_table(df, True)

          n_miss  ratio
Cabin        687  77.10
Age          177  19.87
Embarked       2   0.22


['Age', 'Cabin', 'Embarked']

In [6]:
# na_cols'a kaydediyoruz...
na_cols = missing_values_table(df, True)

          n_miss  ratio
Cabin        687  77.10
Age          177  19.87
Embarked       2   0.22


In [7]:
na_cols

['Age', 'Cabin', 'Embarked']

In [8]:
def missing_vs_target(dataframe, target, na_columns):
    
    # dataframe'in kopyası oluşturuldu, isteğe bağlıdır...
    temp_df = dataframe.copy()

    # bu NA'lerin içerisinde gez dedik:
    for col in na_columns:
        # HAYAT KURTARAN SERİSİ: NA'e sahip olan değişkenlerde gez yanına '_NA_FLAG' şeklinde string bir ekleme yap...
        temp_df[col + '_NA_FLAG'] = np.where(temp_df[col].isnull(), 1, 0)
    
    # Bütün satırları seç fakat içerisinde NA ifadesi olan sütunları getir diyoruz. Bunu list comprehension ile de yapabilirdik.
    na_flags = temp_df.loc[:, temp_df.columns.str.contains("_NA_")].columns

    # na_flags değişkenlerinde gezip bazı işemler yapıyoruz...
    for col in na_flags:
        print(pd.DataFrame({"TARGET_MEAN": temp_df.groupby(col)[target].mean(),
                            "Count": temp_df.groupby(col)[target].count()}), end="\n\n\n")

In [9]:
# Survived değişkeni ile eksikliğe sahip olan bu değişkenleri bir karşılaştır diyoruz...
missing_vs_target(df, "Survived", na_cols)

             TARGET_MEAN  Count
Age_NA_FLAG                    
0               0.406162    714
1               0.293785    177


               TARGET_MEAN  Count
Cabin_NA_FLAG                    
0                 0.666667    204
1                 0.299854    687


                  TARGET_MEAN  Count
Embarked_NA_FLAG                    
0                    0.382452    889
1                    1.000000      2





* Evet üç tane değişkenin bağımlı değişken olan Survived değişkeninin ortalaması açısından değerlendirmeleri geldi... Eksik değere sahip olan noktalara 1 yazdık, doluu olan noktalara 0 yaptık... Şimdi eksik değere sahip olan gözlemlere odaklanıp "Survived" ın ortalamasını aldığımızda 0.29, tam 177 tane eksiklik varmış, yani bu 177 tane eksikliğe sahip olan satır varya bu satırlardaki "Survived" yani hayatta kalma 1, hayatta kalamama 0 durumunu ifade eden değişkendeki bu değişkenin ortalaması 0.29'dur. Yani yaş değişkeninde eksiklik olması senaryosunun hayatta kalma oranı 0.29, yaş değişkeninde dolu olan senaryonun hayatta kalma oranı 0.40. Görüldüğü gibi arada birr fark var gibi gözüküyor. Bundan bir kuşkulandır. İyi de neysen kuşkulandık diye sorulacak olursa: Burada ki ana amacımız bu survived yani hayatta kalma kalmama durumunu tanımlamaya çalışmak. Yani bir modelleme aşamasına geçildiğinde amacımız şu olacak: İnsanların bu gemiye bindiğinde hayatta kalma ya da hayatta kalamama durumlarını ortaya çıkaran şey nedir sorusuna yanıt vermeye çalışacağız ve bazı insanların bilgileri verildiğinde hayatta kalıp kalamayacaklarını tahmin ediyor olacağız. Dolayısıyla ana odağımız survived'a göre survived hayatta kalma durumunu ne etkiliyor, ne ortaya çıkarıyor bunu yakalamaya çalışmaktır. Dolayısıyla buradan hareketle bir gözlemlediğimizde yaş değişkeninde eksiklik olan senaryonun hayatta kalması daha düşüktür. Ama buradan bir sonuca gitmek zor. Şimdilik bunu göz ardı ediyoruz. Peki başka bir değişkene bakalım... cabin_NA_FLAG değişkeninin %75'inden fazlası eksikti. Bu değişken çöp gibi düşünüp bu değişkeni atabilirdik ama DİKKAT cabin değişkeninde NA olanlar yani diyelim ki Mehmet Bey gemiye bindi ama kabin numarası yok veya Mehmet Bey gemiye bindi demeyelim şöyle diyelim gemide bir Mehmet Bey var bu Mehmet Bey'in cabin numarası yok. Bunun gibi cabin numarası olmayan birçok kişi var. NA dediğimiz cabin'de eksiklik olanlar. cabin değişkeninde eksiklik olanların "Survived" hayatta kalma oranları 0.30, cabin değişkeninde eksiklik olmayanların hayatta kalma oranların %66. Buraya ciddi bir odaklanmak gerekir. Bunun sebebi ne acaba? O gemide çalışan  kişilerin cabin kuru yani gemi görevlilerinin birçoğunun kabinlerinin numarası yokmuş çünkü kabinlerde kalmıyorlar. Kabin numaraları olanlar daha çok belki de tamamına yakını yolcular dolayısıyla öyle bir durum ortaya çıkıyor ki normalde veri bize bunu söylemek istiyor aslında ama biraz çekimser belki kalındığından biraz daha az yaratıcı bakıldığında verinin bize söylemek istediği şey gözden kaçabiliyor. Eğer bu cabin değişkenini dışarıda bıraksaydık bu önemli noktayı kaçırmış olacaktık. İş bilgisi biliniyor olsaydı da bu bilgi biliniyor olacaktı burada güzel başka bir derste var... Yani aslında bu NA ifadelerinin birçoğu gemi çalışanlarına aitmiş. cabin'i NA olanların hepsi gemi çalışanı değil buna dikkat edelim. Biz bu cabin değişkenini silmek yerine bu değişkeni oluştururuz. feature extraction bölümünde bunu bir değişkene  çeviriyor olacağız. Diğer örneğe bakalım... embarked değişkenindeki NA leri de flagledik. NA olanlara 1 olmayanlara 0 dedik.  Zaten 2 tane varmış. Buradaki countları atmamızın sebebi bu. Eksik değer sayısına göre bir değerlendirme imkanı da bulalım istiyoruz. Burada 700 tane var burada 170 tane var yaklaşık, tamamdır bu dikkate değerdir diyoruz. Bu dikkate değer ise bunun survived açısından ortalaması nedir diye baktığımızda bu da yine kayda değer gibi ama diyip analizlerimize devam ettik.  Bilmiyoruz gerçekten yaşta NA olup olmamasıyla hayatta kalıp kalınmaması arasında bir ilişki var mı bilmiyoruz. Ama burada bu  ikisi arasındaki oran çok yüksek olduğu için ve frekanslarda kayda değer olduğundan dolayı sanki bir ilişki varmış gibi  geliyor. Ancak bilemiyoruz şu anda. 